# Chapter 8: Defining ORM Data Models

Note: use models from "ch08_dataclass.ipynb" for later chapters.

Define function to remove database file:

In [834]:
import os

def remove_file(file_name):
    if os.path.exists(file_name):
        os.remove(file_name)
        print(f"{file_name} has been removed.")
    else:
        print(f"{file_name} does not exist.")


In [835]:
remove_file("store.db")

store.db has been removed.


Postponed Evaluation of Annotations:

In [836]:
from __future__ import annotations  # PEP-563

Necessary imports:

In [837]:
import datetime
import enum
from decimal import Decimal
from typing import Annotated

from sqlalchemy import (CheckConstraint, ForeignKey, Index, Numeric, String,
                        create_engine)
from sqlalchemy.ext.associationproxy import AssociationProxy, association_proxy
from sqlalchemy.orm import (DeclarativeBase, Mapped, MappedAsDataclass,
                            mapped_column, query_expression, relationship,
                            sessionmaker)

Database URL and engine settings:

In [838]:
DATABASE_URL = "sqlite+pysqlite:///store.db"

In [839]:
engine = create_engine(
    DATABASE_URL,
    echo=True,
)

Base class for declarative mapping:

In [840]:
class Base(DeclarativeBase):
    pass

In [841]:
Base.metadata

MetaData()

Defining the product table:

In [842]:
from sqlalchemy.types import Integer

In [843]:
class ProductType(enum.Enum):
    PHONE = 0
    ACCESSORY = 1
    OTHER = 2

In [844]:
class Product(Base):
    __tablename__ = "product"

    product_id: Mapped[int] = mapped_column(
        "id",
        Integer,
        primary_key=True,
    )
    product_name: Mapped[str] = mapped_column(
        String(255),
        index=True,
    )
    unit_price: Mapped[Decimal] = mapped_column(
        Numeric(12, 2),
        CheckConstraint("unit_price>0"),
    )
    units_in_stock: Mapped[int] = mapped_column(
        CheckConstraint("units_in_stock>=0"),
        default=0,
    )
    type: Mapped[ProductType] = mapped_column(
        default=ProductType.OTHER,
    )

The automatically generated `Table` object:

In [845]:
Product.__table__

Table('product', MetaData(), Column('id', Integer(), table=<product>, primary_key=True, nullable=False), Column('product_name', String(length=255), table=<product>, nullable=False), Column('unit_price', Numeric(precision=12, scale=2), CheckConstraint(<sqlalchemy.sql.elements.TextClause object at 0x7f1612deeb90>), table=<product>, nullable=False), Column('units_in_stock', Integer(), CheckConstraint(<sqlalchemy.sql.elements.TextClause object at 0x7f1612def100>), table=<product>, nullable=False, default=ScalarElementColumnDefault(0)), Column('type', Enum('PHONE', 'ACCESSORY', 'OTHER', name='producttype'), table=<product>, nullable=False, default=ScalarElementColumnDefault(<ProductType.OTHER: 2>)), schema=None)

Create the table with `Metadata.create_all()`:

In [846]:
Base.metadata.create_all(engine)

2024-03-25 20:52:21,312 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-25 20:52:21,313 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("product")
2024-03-25 20:52:21,313 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-25 20:52:21,316 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("product")
2024-03-25 20:52:21,316 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-25 20:52:21,318 INFO sqlalchemy.engine.Engine 
CREATE TABLE product (
	id INTEGER NOT NULL, 
	product_name VARCHAR(255) NOT NULL, 
	unit_price NUMERIC(12, 2) NOT NULL CHECK (unit_price>0), 
	units_in_stock INTEGER NOT NULL CHECK (units_in_stock>=0), 
	type VARCHAR(9) NOT NULL, 
	PRIMARY KEY (id)
)


2024-03-25 20:52:21,319 INFO sqlalchemy.engine.Engine [no key 0.00074s] ()
2024-03-25 20:52:21,322 INFO sqlalchemy.engine.Engine CREATE INDEX ix_product_product_name ON product (product_name)
2024-03-25 20:52:21,323 INFO sqlalchemy.engine.Engine [no key 0.00046s] ()
2024-03-25 20:52:21,325 INFO sqlalchemy.eng

Let's remove the database file to test other settings:

In [847]:
engine.dispose()

In [848]:
remove_file("store.db")

store.db has been removed.


Customizing the type map (integer to small integer):

In [849]:
from sqlalchemy.types import SmallInteger

Redefine the base class:

In [850]:
class Base(DeclarativeBase):
    type_annotation_map = {
        int: SmallInteger,
    }

Using the new base class (note that we're using `server_default` for `units_in_stock` here):

In [851]:
class Product(Base):
    __tablename__ = "product"

    product_id: Mapped[int] = mapped_column(
        "id",
        Integer,
        primary_key=True,
    )

    units_in_stock: Mapped[int] = mapped_column(
        CheckConstraint("units_in_stock>=0"),
        server_default="0.0",
    )

In [852]:
Base.metadata.create_all(engine)

2024-03-25 20:52:21,354 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-25 20:52:21,355 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("product")
2024-03-25 20:52:21,355 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-25 20:52:21,356 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("product")
2024-03-25 20:52:21,356 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-25 20:52:21,357 INFO sqlalchemy.engine.Engine 
CREATE TABLE product (
	id INTEGER NOT NULL, 
	units_in_stock SMALLINT DEFAULT '0.0' NOT NULL CHECK (units_in_stock>=0), 
	PRIMARY KEY (id)
)


2024-03-25 20:52:21,357 INFO sqlalchemy.engine.Engine [no key 0.00028s] ()
2024-03-25 20:52:21,359 INFO sqlalchemy.engine.Engine COMMIT


In [853]:
engine.dispose()

In [854]:
remove_file("store.db")

store.db has been removed.


## Defining Reusable Types with `Annotated`

Defining a reusable type for integer primary keys:

In [855]:
int_pk = Annotated[int, mapped_column(primary_key=True)]

Defining other reusable types:

In [856]:
# date with default set to today
date_auto = Annotated[datetime.date,
                      mapped_column(default=datetime.date.today)]

# timestamp with default set to now
timestamp_auto = Annotated[datetime.datetime,
                           mapped_column(default=datetime.datetime.now)]

# strings with different length
str_127 = Annotated[str, mapped_column(String(127))]
str_255 = Annotated[str, mapped_column(String(255))]

# a number with specified precision and scale
num_12_2 = Annotated[Decimal, mapped_column(Numeric(12, 2))]

Get a new base class:

In [857]:
class Base(DeclarativeBase):
    pass

Product table:

In [858]:
class Product(Base):
    __tablename__ = "product"

    product_id: Mapped[int_pk]
    product_name: Mapped[str_255] = mapped_column(
        index=True,
    )
    unit_price: Mapped[num_12_2] = mapped_column(
        CheckConstraint("unit_price>0"),
    )
    units_in_stock: Mapped[int] = mapped_column(
        CheckConstraint("units_in_stock>=0"),
        default=0,
    )
    type: Mapped[ProductType] = mapped_column(
        default=ProductType.OTHER,
    )

Employee table:

In [859]:
class Employee(Base):
    __tablename__ = "employee"

    employee_id: Mapped[int_pk]

    manager_id: Mapped[int | None] = mapped_column(
        ForeignKey("employee.employee_id"),
        default=None,
    )

    name: Mapped[str_127] = mapped_column(
        CheckConstraint(
            "length(name)>0",
            name="name_length_must_be_at_least_one_character",
        ),
        default="",
    )
    is_manager: Mapped[bool] = mapped_column(default=False)
    hire_date: Mapped[date_auto]

Customer table:

In [860]:
class Customer(Base):
    __tablename__ = "customer"

    customer_id: Mapped[int_pk]

    first_name: Mapped[str_127]
    last_name: Mapped[str_127]
    address: Mapped[str_255]
    email: Mapped[str_127] = mapped_column(unique=True)

    __table_args__ = (
        Index("customer_full_name", "first_name", "last_name"),
    )

Order table:

In [861]:
class Order(Base):
    __tablename__ = "order"

    order_id: Mapped[int_pk]

    customer_id: Mapped[int] = mapped_column(
        ForeignKey("customer.customer_id"),
        default=None,
    )
    employee_id: Mapped[int | None] = mapped_column(
        ForeignKey("employee.employee_id"),
        default=None,
    )

    order_datetime: Mapped[timestamp_auto]
    is_shipped: Mapped[bool] = mapped_column(default=False)

    order_details: Mapped[list[OrderDetail]] = relationship(
        back_populates="order",
        cascade="all, delete-orphan",
        passive_deletes=True,
    )

Order detail table:

In [862]:
class OrderDetail(Base):

    __tablename__ = "order_detail"

    order_id: Mapped[int] = mapped_column(
        ForeignKey("order.order_id", ondelete="CASCADE"),
        primary_key=True,
        default=None,
    )
    product_id: Mapped[int] = mapped_column(
        ForeignKey("product.product_id"),
        primary_key=True,
        default=None,
    )

    quantity: Mapped[int] = mapped_column(
        CheckConstraint(
            "quantity>0",
            name="num_of_ordered_item_must_be_positive",
        ),
        default=1,
    )

Creating the tables:

In [863]:
Base.metadata.create_all(engine)

2024-03-25 20:52:21,415 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-25 20:52:21,415 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("product")
2024-03-25 20:52:21,416 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-25 20:52:21,416 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("product")
2024-03-25 20:52:21,416 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-25 20:52:21,417 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("employee")
2024-03-25 20:52:21,417 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-25 20:52:21,418 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("employee")
2024-03-25 20:52:21,418 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-25 20:52:21,418 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("customer")
2024-03-25 20:52:21,419 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-25 20:52:21,419 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("customer")
2024-03-25 20:52:21,419 INFO sqlalchemy.engine.Engine [raw sql

In [864]:
engine.dispose()

In [865]:
remove_file("store.db")

store.db has been removed.


## Defining Relationships


Start from a clean base:

In [866]:
class Base(DeclarativeBase):
    pass

One-to-many, many-to-many:

In [867]:
class Product(Base):
    __tablename__ = "product"

    product_id: Mapped[int_pk]
    product_name: Mapped[str_255] = mapped_column(
        index=True,
    )
    unit_price: Mapped[num_12_2] = mapped_column(
        CheckConstraint("unit_price>0"),
    )
    units_in_stock: Mapped[int] = mapped_column(
        CheckConstraint("units_in_stock>=0"),
        default=0,
    )
    type: Mapped[ProductType] = mapped_column(
        default=ProductType.OTHER,
    )

    # one-to-many relationship with association objects `OrderDetail`
    order_details: Mapped[list[OrderDetail]] = relationship(
        back_populates="product",
    )

    # many-to-many relationship to `Order`, bypassing `OrderDetail`
    orders: Mapped[list[Order]] = relationship(
        secondary="order_detail",
        back_populates="products",
        viewonly=True,
    )

Self-referential relationship (one-to-many):

In [868]:
class Employee(Base):
    __tablename__ = "employee"

    employee_id: Mapped[int_pk]

    manager_id: Mapped[int | None] = mapped_column(
        ForeignKey("employee.employee_id"),
        default=None,
    )

    name: Mapped[str_127] = mapped_column(
        CheckConstraint(
            "length(name)>0",
            name="name_length_must_be_at_least_one_character",
        ),
        default="",
    )
    is_manager: Mapped[bool] = mapped_column(default=False)
    hire_date: Mapped[date_auto]

    # self-referential relationship between manager and employees
    manager: Mapped[Employee] = relationship(
        back_populates="employees",
        remote_side=[Employee.employee_id],
    )

    employees: Mapped[list[Employee]] = relationship(
        back_populates="manager",
    )

In [869]:
class Customer(Base):
    __tablename__ = "customer"

    customer_id: Mapped[int_pk]

    first_name: Mapped[str_127]
    last_name: Mapped[str_127]
    address: Mapped[str_255]
    email: Mapped[str_127] = mapped_column(unique=True)

    __table_args__ = (
        Index("customer_full_name", "first_name", "last_name"),
    )

    orders: Mapped[list[Order]] = relationship(
        back_populates="customer",
    )

Association proxy (`Order.product_names`):

In [870]:
from sqlalchemy.ext.associationproxy import AssociationProxy, association_proxy

In [871]:
class Order(Base):
    __tablename__ = "order"

    order_id: Mapped[int_pk]

    customer_id: Mapped[int] = mapped_column(
        ForeignKey("customer.customer_id"),
        default=None,
    )
    employee_id: Mapped[int | None] = mapped_column(
        ForeignKey("employee.employee_id"),
        default=None,
    )

    order_datetime: Mapped[timestamp_auto]
    is_shipped: Mapped[bool] = mapped_column(default=False)

    customer: Mapped[Customer] = relationship(
        back_populates="orders",
    )

    # one-to-many relationship with association objects `OrderDetail`
    order_details: Mapped[list[OrderDetail]] = relationship(
        back_populates="order",
        cascade="all, delete-orphan",
        passive_deletes=True,
    )

    # many-to-many relationship with `Product`, bypassing `OrderDetail` class
    products: Mapped[list[Product]] = relationship(
        secondary="order_detail",
        back_populates="orders",
        viewonly=True,  # avoid conflicting changes between relations
    )

    product_names: AssociationProxy[list[str]] = association_proxy(
        "products",
        "product_name",
    )

Association class:

In [872]:
class OrderDetail(Base):

    __tablename__ = "order_detail"

    order_id: Mapped[int] = mapped_column(
        ForeignKey("order.order_id", ondelete="CASCADE"),
        primary_key=True,
        default=None,
    )
    product_id: Mapped[int] = mapped_column(
        ForeignKey("product.product_id"),
        primary_key=True,
        default=None,
    )

    quantity: Mapped[int] = mapped_column(
        CheckConstraint(
            "quantity>0",
            name="num_of_ordered_item_must_be_positive",
        ),
        default=1,
    )

    # many-to-one relationship
    order: Mapped[Order] = relationship(
        back_populates="order_details",
    )

    # many-to-one relationship
    product: Mapped[Product] = relationship(
        back_populates="order_details",
    )

One-to-one relationship (for explanation only, not included in our system):

In [873]:
class Parent(Base):
    __tablename__ = "parent_table"

    id: Mapped[int_pk]

    # one-to-many relationship with only one child
    child: Mapped[Child] = relationship(back_populates="parent")


class Child(Base):
    __tablename__ = "child_table"

    id: Mapped[int_pk]
    parent_id: Mapped[int] = mapped_column(ForeignKey("parent_table.id"))

    # still a many-to-one relationship
    parent: Mapped[Parent] = relationship(back_populates="child")

In [874]:
Base.metadata.create_all(engine)

2024-03-25 20:52:21,501 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-25 20:52:21,502 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("product")
2024-03-25 20:52:21,502 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-25 20:52:21,503 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("product")
2024-03-25 20:52:21,503 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-25 20:52:21,504 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("employee")
2024-03-25 20:52:21,504 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-25 20:52:21,505 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("employee")
2024-03-25 20:52:21,505 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-25 20:52:21,505 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("customer")
2024-03-25 20:52:21,506 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-25 20:52:21,506 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("customer")
2024-03-25 20:52:21,506 INFO sqlalchemy.engine.Engine [raw sql